In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.multioutput import ClassifierChain

example_submit = pd.read_csv('HeadHunter_sample_submit.csv')
train = pd.read_csv('HeadHunter_train_prep.csv')
test = pd.read_csv('HeadHunter_test_prep.csv')

In [3]:
for feature in ['position', 'negative', 'positive']:
    train[feature] = train[feature].fillna('')
    test[feature] = test[feature].fillna('')

In [4]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer, StandardScaler, MinMaxScaler, RobustScaler, PolynomialFeatures
from sklearn.pipeline import make_pipeline, Pipeline, FeatureUnion
from sklearn.compose import ColumnTransformer
import copy
from sklearn.compose import ColumnTransformer

from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split

cv = StratifiedKFold(n_splits=5)

def get_feature_selector(features):
    pipeline = Pipeline([
        ("selector", ColumnTransformer([
            ("selector", "passthrough", features)
        ], remainder="drop"))
    ])
    return pipeline

features = ['salary_rating', 'team_rating', 'managment_rating', 'career_rating',
       'workplace_rating', 'rest_recovery_rating', 'population',
       'children', 'latitude_dd', 'longitude_dd', 'positive_len',
       'negative_len', 'position_len', 'positive_contains_gmail',
       'negative_contains_gmail', 'positive_contains_commarystarkova',
       'negative_contains_commarystarkova', 'edit_d', 'pos_has_name', 'neg_has_name']

features = features + ['positive_contains_ссылка', 'negative_contains_ссылка', 'positive_contains_адрес','negative_contains_адрес']

features = features + ['negative_dost:neutral',
       'negative_dost:skip', 'negative_dost:negative',
       'negative_dost:positive', 'negative_dost:speech',
       'positive_dost:neutral', 'positive_dost:skip', 'positive_dost:negative',
       'positive_dost:positive', 'positive_dost:speech']

In [5]:
Y = np.stack([train.target.apply(lambda s: int(str(i) in s)).values for i in range(9)]).T

In [6]:
train = train[features + ['position', 'positive', 'negative']]

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
from sklearn.metrics import make_scorer

In [9]:
text_trans = ColumnTransformer(
    [('position_transformer', CountVectorizer(), 'position'),
     ('positive_transformer', CountVectorizer(), 'positive'),
     ('negative_transformer', CountVectorizer(), 'negative')],
    remainder='drop')#verbose_feature_names_out=False

union = FeatureUnion([('textual_transformer', text_trans),
                      ('features', get_feature_selector(features))])

xgb = XGBClassifier(n_estimators=500, max_depth=3, verbosity=0, tree_method='gpu_hist')
clf = Pipeline([('transform', union), ('clf', ClassifierChain(xgb, order=[8, 3, 1, 7, 6, 5, 2, 4, 0], cv=cv))])

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(train, Y, test_size=0.3)
#clf.fit(X_train, y_train).predict(X_test)

cross_val_score(clf, train, Y, scoring=make_scorer(f1_score, average='samples'), cv=5, verbose=2)

In [10]:
def get_score(thresholds):
    y_true = Y
    y_pred = np.stack([predictions[::, i] > thresholds[i] for i in range(9)]).T.astype(int)
    return f1_score(y_pred, y_true, average='samples')

In [13]:
import optuna
def objective(trial):
    thresholds = {0: trial.suggest_float('0', 0.0, 1.0),
     1: trial.suggest_float('1', 0.0, 1.0),
     2: trial.suggest_float('2', 0.0, 1.0),
     3: trial.suggest_float('3', 0.0, 1.0),
     4: trial.suggest_float('4', 0.0, 1.0),
     5: trial.suggest_float('5', 0.0, 1.0),
     6: trial.suggest_float('6', 0.0, 1.0),
     7: trial.suggest_float('7', 0.0, 1.0),
     8: trial.suggest_float('8', 0.0, 1.0),}
    return get_score(thresholds)

In [22]:
from tqdm import tqdm
xgb = XGBClassifier(n_estimators=500, max_depth=3, verbosity=0, tree_method='gpu_hist')
clf = Pipeline([('transform', union), ('clf', ClassifierChain(xgb, order=[8, 0, 4, 3, 1, 7, 6, 5, 2], cv=cv))])
predictions = cross_val_predict(clf, train, Y, cv=5, verbose=2, method='predict_proba')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=1000)
best_thresholds = study.best_params

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 10.6min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 52.9min finished
[I 2022-02-19 14:59:16,835] A new study created in memory with name: no-name-74a7d758-86f3-4bf3-ad4d-091b97a41293
[I 2022-02-19 14:59:16,939] Trial 0 finished with value: 0.3776115784783919 and parameters: {'0': 0.9933736893867724, '1': 0.8517957080923951, '2': 0.9669988148187494, '3': 0.6558358158233756, '4': 0.5830409763580788, '5': 0.31937304130302635, '6': 0.19539217105878948, '7': 0.3021051645384213, '8': 0.8792415111978774}. Best is trial 0 with value: 0.3776115784783919.
[I 2022-02-19 14:59:17,018] Trial 1 finished with value: 0.703627119532458 and parameters: {'0': 0.9767656079350349, '1': 0.6557075236813495, '2': 0.4977639012481614, '3': 0.336113665506767, '4': 0.7718318513931582, '5': 0.9205641849700577, '6': 0.31177451139532464, '7': 0.910556803377

[I 2022-02-19 14:59:18,600] Trial 21 finished with value: 0.8305214545917431 and parameters: {'0': 0.3293131674827281, '1': 0.23344281526987729, '2': 0.006674755202188631, '3': 0.16894219691803686, '4': 0.0010037122817559072, '5': 0.8342235597847436, '6': 0.21317101173732922, '7': 0.7649806107116662, '8': 0.29212511611839426}. Best is trial 16 with value: 0.8331865712713264.
[I 2022-02-19 14:59:18,683] Trial 22 finished with value: 0.8200564210274094 and parameters: {'0': 0.11849252043823769, '1': 0.23867421752702392, '2': 0.16515584284312895, '3': 0.26618656561148024, '4': 0.012087296339838738, '5': 0.7991032225680985, '6': 0.18392610753639937, '7': 0.9544465328146713, '8': 0.3028024927417668}. Best is trial 16 with value: 0.8331865712713264.
[I 2022-02-19 14:59:18,766] Trial 23 finished with value: 0.8249357915978718 and parameters: {'0': 0.2577943312780707, '1': 0.14890237907341758, '2': 0.1956696415715117, '3': 0.37830343098335784, '4': 0.09673113461012894, '5': 0.8433965896246572,

[I 2022-02-19 14:59:20,607] Trial 43 finished with value: 0.8331590533847 and parameters: {'0': 0.1976047638652765, '1': 0.2648010509476541, '2': 0.1727938614470175, '3': 0.21586854815255413, '4': 0.03561240902822252, '5': 0.7847859413675924, '6': 0.3148377235161607, '7': 0.8328923361671468, '8': 0.32463380397570096}. Best is trial 42 with value: 0.8352019288728149.
[I 2022-02-19 14:59:20,685] Trial 44 finished with value: 0.8262835128547842 and parameters: {'0': 0.1885682015288601, '1': 0.158555689547952, '2': 0.31328835192491855, '3': 0.20619815364769678, '4': 0.03975092496772967, '5': 0.7874047329629627, '6': 0.31156579524145644, '7': 0.7231764123288179, '8': 0.44556776556985944}. Best is trial 42 with value: 0.8352019288728149.
[I 2022-02-19 14:59:20,771] Trial 45 finished with value: 0.8338607594936709 and parameters: {'0': 0.2532392331278128, '1': 0.24701669397816373, '2': 0.6095847660635608, '3': 0.5982034148044109, '4': 0.16749638344677323, '5': 0.8663596983248335, '6': 0.11930

[I 2022-02-19 14:59:22,648] Trial 65 finished with value: 0.80611018760834 and parameters: {'0': 0.0906567206515902, '1': 0.2488663959117994, '2': 0.6246433583696517, '3': 0.028004513074935483, '4': 0.08190896460194257, '5': 0.8367759456839345, '6': 0.1830682883607751, '7': 0.8935091351312264, '8': 0.4239476988795493}. Best is trial 54 with value: 0.8356101108577717.
[I 2022-02-19 14:59:22,735] Trial 66 finished with value: 0.7733704544008027 and parameters: {'0': 0.3278498001783952, '1': 0.123057146155699, '2': 0.6705248452099776, '3': 0.1963515733997533, '4': 2.8868714604243437e-05, '5': 0.9663251439044385, '6': 0.21174602798579517, '7': 0.8359506131867117, '8': 0.47995871843326493}. Best is trial 54 with value: 0.8356101108577717.
[I 2022-02-19 14:59:22,824] Trial 67 finished with value: 0.8210654102037073 and parameters: {'0': 0.1376932510089058, '1': 0.20495832653439255, '2': 0.7557888855539696, '3': 0.6571965534856602, '4': 0.13178853766897475, '5': 0.7117208094920882, '6': 0.275

[I 2022-02-19 14:59:24,607] Trial 87 finished with value: 0.8302736064155987 and parameters: {'0': 0.279612310908246, '1': 0.15706902302982934, '2': 0.5409868281729184, '3': 0.024482753623734116, '4': 0.26977239098233285, '5': 0.6791580439163916, '6': 0.3016062559339611, '7': 0.6857928365837835, '8': 0.25196717719157}. Best is trial 84 with value: 0.8362181251146578.
[I 2022-02-19 14:59:24,728] Trial 88 finished with value: 0.8341686977487748 and parameters: {'0': 0.26122850438337114, '1': 0.19698687072137055, '2': 0.6824324410148366, '3': 0.2226241147187031, '4': 0.14896196456409855, '5': 0.7747613566051191, '6': 0.37196853088078485, '7': 0.6428214266542776, '8': 0.1737237918601134}. Best is trial 84 with value: 0.8362181251146578.
[I 2022-02-19 14:59:24,845] Trial 89 finished with value: 0.7042921351259269 and parameters: {'0': 0.3091313297582536, '1': 0.28912719080881283, '2': 0.7828600870184348, '3': 0.28465209072838954, '4': 0.06967937130193602, '5': 0.7086913762481353, '6': 0.688

...

[I 2022-02-19 15:01:21,030] Trial 901 finished with value: 0.8359002654446478 and parameters: {'0': 0.22976448734838706, '1': 0.39532467001401794, '2': 0.38187011997147197, '3': 0.29840416011573556, '4': 0.839347056625317, '5': 0.19964007387108634, '6': 0.14318986250387417, '7': 0.1701783246084155, '8': 0.24990074888443342}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:21,203] Trial 902 finished with value: 0.8370768890935569 and parameters: {'0': 0.2698198307207768, '1': 0.36734198158778203, '2': 0.3240545483490682, '3': 0.23947107687433897, '4': 0.7767830431367815, '5': 0.1798177438073393, '6': 0.29356141650151285, '7': 0.24549570480524374, '8': 0.2809671923974494}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:21,515] Trial 903 finished with value: 0.8357944994178188 and parameters: {'0': 0.25005293974753307, '1': 0.3311844855341792, '2': 0.42597900770977826, '3': 0.17328086134617535, '4': 0.8005275265192981, '5': 0.3235466014473994, 

[I 2022-02-19 15:01:25,294] Trial 923 finished with value: 0.835974863252951 and parameters: {'0': 0.227091922518005, '1': 0.3654820125536823, '2': 0.36028558666762206, '3': 0.2811536408092802, '4': 0.8074211332109419, '5': 0.2600870910256384, '6': 0.20465439357391027, '7': 0.16784647717183845, '8': 0.23498372872399886}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:25,476] Trial 924 finished with value: 0.8366692686981231 and parameters: {'0': 0.2751381189257461, '1': 0.29869455294592406, '2': 0.26404567526597733, '3': 0.34556072983222474, '4': 0.8331152322441807, '5': 0.22973480021758247, '6': 0.1534363564978335, '7': 0.24928257371547646, '8': 0.2953401225125808}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:25,776] Trial 925 finished with value: 0.837070150019281 and parameters: {'0': 0.24599122937146223, '1': 0.24370078900892314, '2': 0.4000142381428847, '3': 0.25392552113668887, '4': 0.5152918573422129, '5': 0.29003745877583864, '6'

[I 2022-02-19 15:01:29,308] Trial 945 finished with value: 0.8346759066862849 and parameters: {'0': 0.20652170745519138, '1': 0.291902365865697, '2': 0.18243144301312098, '3': 0.2810797239397371, '4': 0.07345759479494295, '5': 0.1194678248056239, '6': 0.3462316436059325, '7': 0.21802471317579064, '8': 0.25056088445415214}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:29,477] Trial 946 finished with value: 0.8366621552308319 and parameters: {'0': 0.2737821065799098, '1': 0.4059911024883071, '2': 0.25241043893225984, '3': 0.2401816280823953, '4': 0.7420772916488138, '5': 0.26119071772887564, '6': 0.208442791608195, '7': 0.15249748040961686, '8': 0.27932831406985864}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:29,714] Trial 947 finished with value: 0.8362116668351434 and parameters: {'0': 0.2310209679053917, '1': 0.33287816160743816, '2': 0.2827366259055471, '3': 0.2892752478247268, '4': 0.758922916219102, '5': 0.09578951133207338, '6': 

[I 2022-02-19 15:01:33,647] Trial 967 finished with value: 0.8370747363337188 and parameters: {'0': 0.2721886348442877, '1': 0.24831286543583125, '2': 0.20253558015532835, '3': 0.24610733208552257, '4': 0.11573660084505388, '5': 0.24258941525801297, '6': 0.24245270386664036, '7': 0.27066792257153227, '8': 0.2534133183692013}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:33,818] Trial 968 finished with value: 0.8351956577898083 and parameters: {'0': 0.30447757463204017, '1': 0.2102292199809954, '2': 0.17666178125076853, '3': 0.21274174310990696, '4': 0.23229311551255888, '5': 0.21383535968993078, '6': 0.20474525017283202, '7': 0.9284636125106644, '8': 0.22829000654332368}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:33,980] Trial 969 finished with value: 0.837110116473667 and parameters: {'0': 0.2720427099069427, '1': 0.2370235307167285, '2': 0.2251682098891955, '3': 0.2513026894258892, '4': 0.14293962001425084, '5': 0.2803717895019567,

[I 2022-02-19 15:01:38,173] Trial 989 finished with value: 0.8356634618624554 and parameters: {'0': 0.34201584334358737, '1': 0.28902224917696095, '2': 0.23204118872348103, '3': 0.19843277225303912, '4': 0.3626571600154635, '5': 0.2813952387290587, '6': 0.2557217907819636, '7': 0.2298313079910982, '8': 0.24516427990539524}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:38,346] Trial 990 finished with value: 0.8358405497587038 and parameters: {'0': 0.2990513448431072, '1': 0.18526503234061692, '2': 0.22068874322767157, '3': 0.26972528725744427, '4': 0.15301898546035386, '5': 0.33002841541664785, '6': 0.28182585333950966, '7': 0.24700650678574615, '8': 0.26518371429579224}. Best is trial 792 with value: 0.8373461712698288.
[I 2022-02-19 15:01:38,532] Trial 991 finished with value: 0.8365296201034073 and parameters: {'0': 0.3166549670350628, '1': 0.3039596902259245, '2': 0.23721490743274604, '3': 0.2075170422954874, '4': 0.1771706426582058, '5': 0.2249454048155873,

In [25]:
thresholds = study.best_params
y_true = Y
y_pred = np.stack([predictions[::, i] > thresholds[str(i)] for i in range(9)]).T.astype(int)
f1_score(y_pred, y_true, average='samples')

0.8373461712698288

In [26]:
thresholds

{'0': 0.2590462638347055,
 '1': 0.3189814247506846,
 '2': 0.37260181749549115,
 '3': 0.29589715312553283,
 '4': 0.761422884747047,
 '5': 0.30548755961442997,
 '6': 0.23334675447858247,
 '7': 0.2319297719860448,
 '8': 0.26461844523688727}

In [ ]:
pd.DataFrame([thresholds]).to_csv('tresholds_xgb.csv')

In [27]:
from copy import deepcopy
y_pred_mod = deepcopy(y_pred)

for i in range(len(y_pred_mod)):
    if np.sum(y_pred_mod[i]) == 0:
        y_pred_mod[i][0] = 1

f1_score(y_pred_mod, y_true, average='samples')

0.838053774068791

In [28]:
xgb

XGBClassifier(base_score=None, booster=None, colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=None, gamma=None,
              gpu_id=None, importance_type='gain', interaction_constraints=None,
              learning_rate=None, max_delta_step=None, max_depth=3,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=500, n_jobs=None, num_parallel_tree=None,
              objective='binary:logistic', random_state=42, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=None, subsample=None,
              tree_method='gpu_hist', validate_parameters=None, verbosity=0)

In [29]:
clf.fit(train, Y)

Pipeline(memory=None,
         steps=[('transform',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('textual_transformer',
                                                 ColumnTransformer(n_jobs=None,
                                                                   remainder='drop',
                                                                   sparse_threshold=0.3,
                                                                   transformer_weights=None,
                                                                   transformers=[('position_transformer',
                                                                                  CountVectorizer(analyzer='word',
                                                                                                  binary=False,
                                                                                                  decode_error='strict',
                                

In [ ]:
train.shape, test.shape

In [ ]:
train.columns

In [ ]:
train_predictions = clf.predict_proba(train)
train_predictions = np.stack([train_predictions[::, i] > thresholds[str(i)] for i in range(9)]).T.astype(int)

test_predictions_prob = clf.predict_proba(test[list(train.columns)])
test_predictions = np.stack([test_predictions_prob[::, i] > thresholds[str(i)] for i in range(9)]).T.astype(int)

In [ ]:
test_predictions_prob.shape

In [ ]:
test[['review_id']]

In [ ]:
pd.concat([test[['review_id']], pd.DataFrame(test_predictions_prob, columns = ['0', '1', '2', '3', '4', '5', '6', '7', '8'])], axis=1).to_csv('xgb_probs_test.csv')

In [ ]:
y_true_train = Y
y_pred_train = train_predictions
f1_score(y_pred_train, y_true_train, average='samples')

In [ ]:
y_pred_mod = deepcopy(y_pred_train)
for i in range(len(y_pred_mod)):
    if np.sum(y_pred_mod[i]) == 0:
        y_pred_mod[i][0] = 1
f1_score(y_pred_mod, y_true_train, average='samples')

In [ ]:
y_pred_test = test_predictions
y_pred_test_mod = deepcopy(y_pred_test)
for i in range(len(y_pred_test_mod)):
    if np.sum(y_pred_test_mod[i]) == 0:
        y_pred_test_mod[i][0] = 1

In [ ]:
np.sum(y_pred_test_mod, axis=0)

In [ ]:
np.sum(y_pred_test_mod, axis=0)

In [ ]:
target_column = list()
for i in range(len(y_pred_test_mod)):
    target_column.append(','.join(map(str, np.where(y_pred_test_mod[i])[0])))

example_submit['target'] = target_column
example_submit.to_csv("submits/xgb_500_3_hyppar_tunned_multichain.csv", index=False)